Run the cell below to import the code we'll use in this notebook:

In [ ]:
from datascience import *
import numpy as np
import matplotlib.pyplot as plots
%matplotlib inline
import statsmodels.formula.api as smf
plots.style.use('fivethirtyeight')

# Correlation, regression, and prediction

One of the most important and interesting aspects of data science is making predictions about the future. How can we learn about temperatures a few decades from now by analyzing historical data about climate change and pollution? Based on a person's social media profile, what conclusions can we draw about their interests? How can we use a patient's medical history to judge how well he or she will respond to a treatment?

In this module, you will look at two **correlated** phenomena and predict unseen data points!

We will be using data from the online data archive of Prof. Larry Winner of the University of Florida. The file *hybrid* contains data on hybrid passenger cars sold in the United States from 1997 to 2013. In order to analyze the data, we must first **import** it to our Jupyter notebook and **create a table.**

In [ ]:
hybrid = Table.read_table('http://inferentialthinking.com/notebooks/hybrid.csv')  # Imports the data and creates a table
hybrid.show(5)  # Displays the first five rows of the table

*References: vehicle: model of the car, year: year of manufacture, msrp: manufacturer's suggested retail price in 2013 dollars, acceleration: acceleration rate in km per hour per second, mpg: fuel economy in miles per gallon, class: the model's class.*

**Note: whenever we write an equal sign (=) in python, we are assigning somthing to a variable.**

Let's visualize some of the data to see if we can spot a possible association!

In [ ]:
hybrid.scatter('acceleration', 'msrp') # Creates a scatter plot of two variables in a table

As we can see in the above scatter, there seems to be a positive association between acceleration and price. That is, cars with greater acceleration tend to cost more, on average; conversely, cars that cost more tend to have greater acceleration on average.

What about miles per gallon and price? Do you expect a positive or negative association?

In [ ]:
hybrid.scatter('mpg', 'msrp')

Along with the negative association, the scatter diagram of price versus efficiency shows a **non-linear relation** between the two variables, i.e., the points appear to be clustered around a curve, not around a straight line.

Let's subset the data so that we're only looking at SUVs. Use what you learned in the previous notebook to choose only `SUV`s and then make a scatter plot of `mpg` and `msrp`:

In [ ]:
# task

What do you see?

---

Go ahead and make some more visualizations to get more famliliar with the data here:

In [ ]:
# task

---

### The correlation coefficient - *r*

> The correlation coefficient ranges from −1 to 1. A value of 1 implies that a linear equation describes the relationship between X and Y perfectly, with all data points lying on a line for which Y increases as X increases. A value of −1 implies that all data points lie on a line for which Y decreases as X increases. A value of 0 implies that there is no linear correlation between the variables. ~Wikipedia

*r* = 1: the scatter diagram is a perfect straight line sloping upwards

*r* = -1: the scatter diagram is a perfect straight line sloping downwards.

Let's calculate the correlation coefficient between acceleration and price. We can use the `np.corrcoef` function on the two variable (columns here) that we want to correlate:

In [ ]:
np.corrcoef(hybrid['acceleration'], hybrid['msrp'])

This function returns a matrix for each variable. We have two 1s because, of course, acceleration is correlated perfectly with acceleration. Our coefficient here is 0.6955779, ***implying strong positive correlation***.

---

### Regression

As mentioned earlier, an important tool in data science is to make predictions based on data. The code that we've created so far has helped us establish a relationship between our two variables. Once a relationship has been established, it's time to create a model that predicts unseen data values. To do this we'll find the equation of the **regression line**!

The regression line is the **best fit** line for our data. It’s like an average of where all the points line up. In linear regression, the regression line is a perfectly straight line! Below is a picture showing the best fit line.

![image](http://onlinestatbook.com/2/regression/graphics/gpa.jpg)

As you can infer from the picture, once we find the **slope** and the **y-intercept** we can start predicting values! The equation for the above regression to predict university GPA based on high school GPA would look like this:

$UNIGPA_i= \alpha + \beta HSGPA + \epsilon_i$

The variable we want to predict (or model) is the left side `y` variable, the variable which we think has an influence on our left side variable is on the right side. The $\alpha$ term is the y-intercept and the $\epsilon_i$ describes the randomness.

We can fit the model by setting up an equation without the $\alpha$ and $\epsilon_i$ in the `formula` parameter of the function below, we'll give it our data variable in the `data` parameter. Then we just `fit` the model and ask for a `summary`. We'll try a model for:

$MSRP_i= \alpha + \beta ACCELERATION + \epsilon_i$

In [ ]:
mod = smf.ols(formula='msrp ~ acceleration', data=hybrid.to_df())
res = mod.fit()
print(res.summary())

That's a lot of information! While we should consider everything, we'll look at the `p` value, the `coef`, and the `R-squared`. A p-value of < .05 is generally considered to be statistically significant. The `coef` is how much increase one sees in the left side variable for a one unit increase of the right side variable. So for a 1 unit increase in acceleration one might see an increase of $5067 MSRP, according to our model. But how great is our model? That's the `R-squared`. The `R-squared` tells us how much of the variation in the data can be explained by our model, .484 isn't that bad, but obviously more goes into the MSRP value of a car than *just* acceleration.

We can plot this line of "best fit" too:

In [ ]:
hybrid.scatter('acceleration', 'msrp', fit_line=True)

### Prediction

We can also use this model to predict the MSRP of new data. Say we found hybrids with acceleration of 8, 9, and 10 respectively:

In [ ]:
res.predict({'acceleration': [8,9,10]})

That's it! By working through this module, you've learned how to **visually analyze your data**, **establish a correlation** by calculating the **correlation coefficient**, **find the regression line** and **predict data points**!

---

***We would also appreciate if you filled out this feedback form regarding the notebook:
https://goo.gl/forms/ADY9TJU3TGKlllyT2***

***Your input allows us to continue improving our educational notebooks!***

---

---


## Using your own data

For your project you'll have to `import` data files, merge tables, and then do a short analysis. We've set up some code for you to do that.

To upload your data make sure you're in the directory with the green notebook:

![upload](images/upload.png)

Click the "Upload" button and upload your CSV. You should see it appear in the directory:

![uploaded](images/uploaded.png)

Now you put the name of your file in the cell below. Make sure to run the cell after your make the changes

In [ ]:
my_data = Table.read_table('YOUR-FILE-NAME.csv')
my_data

If you're using more than one data file, read in the next data file below, otherwise delete the cell:

In [ ]:
my_data2 = Table.read_table('YOUR-FILE-NAME.csv')
my_data2

Look back at the previous notebook to figure out how to subset and join your data for analysis:

In [ ]:
# work on subsetting and joining your data here

To find an association between two variables, the `.scatter` method is perhaps the most useful one. 
Try creating a few scatter plots of variables you might think are related among your data!

In [ ]:
# create some visualizations of your data here

Calculate a correlation coefficient on your data!

In [ ]:
# calculate the correlation coefficient here

Run a simple regression on your data

In [ ]:
# run a regression here